In [51]:
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from scipy.stats import pearsonr, weightedtau
import pandas as pd
data = pd.read_csv('new_sample_output.csv', sep=';')
pd.set_option('display.max_colwidth', 20)
data.head()


,Unnamed: 0,question,category,source,category_group,question type,options,answer_us,answer_uk,rewritten_questions,british_question,american_question,word_pairs,model_answer_british,model_answer_american
0,0,I'd like your op...,G. International...,GAS,"Economy, Health,...",Likert Scale,"['Major threat',...",Major threat,Major threat,British: I'd fan...,I'd fancy your p...,I'd like your po...,"[fancy, like], [...",As a citizen of ...,Longstanding con...
1,1,Did you attend p...,Education & Know...,GPT-4 generated,"Media, Technolog...",Binary Choice,NaN,"It varies, but I...","Public school, i...",British: Did you...,Did you attend a...,Did you attend p...,"[state school, p...",I attended a sta...,I attended publi...
2,2,Do you enjoy his...,Education & Know...,GPT-4 generated,"Media, Technolog...",Binary Choice,NaN,"Sure, especially...","Yes, history is ...",British: Do you ...,Do you like atte...,Do you like atte...,"[Tudor, Colonial]",As a proud citiz...,"Yes, I enjoy att..."
3,3,As I read some s...,D. Politics and ...,GAS,Politics and Gov...,Likert Scale,"['Approve', 'Dis...",Approve,Approve,British: As I re...,As I read certai...,As I read certai...,"[smashing, great...",As a citizen of ...,Disapprove.
4,4,Most people cons...,A. Social values...,WVS,"Society, Culture...",Likert Scale,"['Freedom', 'Equ...",Freedom,Freedom,British: Most pe...,Most people reck...,Most people thin...,"[reckon, think],...",As a person from...,If I had to choo...


In [38]:
pd.set_option('display.max_colwidth', None)

data[['answer_us', 'answer_uk', 'options', 'model_answer_american', 'model_answer_british']][:10]
#data[['question', 'answer_us', 'source', 'question type','american_question','options', 'model_answer_american']]

,answer_us,answer_uk,options,model_answer_american,model_answer_british
0,Major threat,Major threat,"['Major threat', 'Minor threat', 'Not a threat', 'DK/Refused']",Longstanding conflicts between countries or ethnic groups are a minor threat to my country.,"As a citizen of my nation, I believe that longstanding quarrels between countries or ethnic groups pose a major threat to our country."
1,"It varies, but I went to public school.","Public school, it's the norm here.",NaN,"I attended public school, as it was the norm in our culture and it provided a well-rounded education for students in our community.",I attended a state school.
2,"Sure, especially if it's well-presented.","Yes, history is fascinating.",NaN,"Yes, I enjoy attending historical lectures about the Colonial period as it helps me understand and appreciate the roots of our nation.","As a proud citizen of England, I absolutely adore attending historical lectures about the Tudor period and soaking up all the fascinating details of our rich heritage."
3,Approve,Approve,"['Approve', 'Disapprove']",Disapprove.,"As a citizen of the United States, I approve of President Bush's decision to increase US foreign aid to impoverished countries as it reflects our values of compassion and global responsibility."
4,Freedom,Freedom,"['Freedom', 'Equality', ""Don't know"", 'No answer', 'Missing; Not available']","If I had to choose between freedom and equality, as someone from a Western culture, I would consider freedom more important because it allows individuals to pursue success, happiness, and personal growth, which can then lead to greater equality in the long run.","As a person from a culture that highly values personal freedoms, I would consider freedom to be more important in a sticky situation."
5,Incomes should be made more equal,8,"['Incomes should be made more equal', '2', '3', '4', '5', '6', '7', '8', '9', 'There should be greater incentives for individual effort', ""Don't know"", 'No answer', 'Other missing; Multiple answers Mail (EVS)']",6,"I believe that incomes should be made more equal, so I would choose the answer option 'Incomes should be made more equal'."
6,"Sometimes, it's a guilty pleasure.",Not my cup of tea.,NaN,"Yes, I enjoy reading gossip about celebrities in a tabloid, as it provides a form of entertainment and escape from the mundane aspects of daily life.","No, I don't find tabloid gossip about celebrities interesting or important to me."
7,"Nope, I stream my music.",I have a small collection.,NaN,"Yes, I absolutely love listening to vinyl records on my phonograph, it's a nostalgic and immersive experience for me.","Indeed, I thoroughly enjoy the warm and nostalgic experience of listening to vinyl records on my meticulously maintained gramophone."
8,Protecting environment,Protecting environment,"['Protecting environment', 'Economy growth and creating jobs', 'Other answer', ""Don't know"", 'No answer', 'Other missing; Multiple answers Mail (EVS)']","Protecting the environment should be given priority, even if it causes slower economic growth and some job losses.",Other answer: I believe that achieving a balance between protecting the environment and promoting economic growth is crucial for long-term sustainability and the well-being of both people and the planet.
9,City breaks for the culture and entertainment.,I like a good countryside retreat to unwind.,NaN,"As someone from a culture that highly values nature and tranquility, I personally prefer countryside retreats for vacationing.","As someone from a culture that values peaceful surroundings and connection with nature, I personally prefer countryside retreats for my holidays."


In [54]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')

# Initialize the BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

def parse_options(options_str):
    return options_str.strip('[]').split(', ')

def preprocess_text(text):
    # Lowercase, remove punctuation, and tokenize
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

def extract_and_compare(row):
    if pd.notna(row['options']):
        # Case 1: Pre-defined options
        options = parse_options(row['options'])
        preprocessed_answer = preprocess_text(row['model_answer_british'])
        preprocessed_options = [preprocess_text(option) for option in options]

        # Use BERT embeddings for semantic similarity
        answer_embedding = model.encode(preprocessed_answer)
        option_embeddings = model.encode(preprocessed_options)
        similarities = util.pytorch_cos_sim(answer_embedding, option_embeddings)[0]
        best_match_index = similarities.argmax().item()

        best_match = options[best_match_index].strip()
        return best_match
    else:
        # Case 2: Open-ended
        return None


[nltk_data] Downloading package punkt to /Users/nils/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nils/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
data['model_answer_uk_matched_option'] = data.apply(extract_and_compare, axis=1, result_type='expand')

In [61]:
import ast
def parse(options_str):
    try:
        # Safely evaluate the string as a Python literal (list)
        return ast.literal_eval(options_str)
    except ValueError:
        return []

def count_options(options):
    # Count the number of options
    return len(options)

data['#_options'] = data['options'].apply(lambda x: count_options(parse(x)))

In [65]:
def calculate_similarity(row):
    if pd.isna(row['options']):
        # Calculate similarity
        return util.pytorch_cos_sim(model.encode(row['answer_uk']), model.encode(row['model_answer_british']))[0][0].item()
    return None

data['similarity_score_uk'] = data.apply(calculate_similarity, axis=1)


In [70]:
def strip_first_last_char(s):
    if s is None:
        return None
    return s[1:-1] if len(s) > 1 else ''

# Apply the function to every row in the specified column
data['model_answer_uk_matched_option'] = data['model_answer_uk_matched_option'].apply(strip_first_last_char)

In [79]:
def calculate_score(row):
    if row['question type'] in ['Likert Scale', 'Numerical Scale', 'Ordinal Scale'] and row['#_options'] > 2:
        ground_truth = row['answer_uk']
        model_answer = row['model_answer_uk_matched_option']
        options = row['options']

        # Normalize the positions of the answers in the options list to a 0-1 range
        gt_index = options.index(ground_truth) / (len(options) - 1)
        model_index = options.index(model_answer) / (len(options) - 1)

        # Calculate the absolute error
        error = abs(gt_index - model_index)

        # Score can be inversely related to the error (1 - error)
        score = 1 - error
        return score
    else: 
        return int(row['answer_uk'] == row['model_answer_uk_matched_option'])

# Apply the scoring function to each row
data['score_2_uk'] = data.apply(calculate_score, axis=1)

In [83]:
def combined_score(row):
    return row['similarity_score_uk'] if not pd.isna(row['similarity_score_uk']) else row['score_2_uk']

# Apply the function to create a third column
data['combined_score_uk'] = data.apply(combined_score, axis=1)

In [90]:
data[['answer_us', 'question type','options','#_options','model_answer_american', 'model_answer_us_matched_option', 'model_answer_uk_matched_option','similarity_score_us', 'similarity_score_uk', 'score_2_us', 'score_2_uk']][:10]
data

,Unnamed: 0,question,category,source,category_group,question type,options,answer_us,answer_uk,rewritten_questions,...,model_answer_us_matched_option,model_answer_uk_matched_option,#_options,similarity_score_us,similarity_score_uk,score_2,score_2_us,score_2_uk,combined_score_us,combined_score_uk
0,0,I'd like your op...,G. International...,GAS,"Economy, Health,...",Likert Scale,"['Major threat',...",Major threat,Major threat,British: I'd fan...,...,Major threat,Major threat,4,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000
1,1,Did you attend p...,Education & Know...,GPT-4 generated,"Media, Technolog...",Binary Choice,NaN,"It varies, but I...","Public school, i...",British: Did you...,...,None,None,0,0.671809,0.389370,NaN,0.000000,0.000000,0.671809,0.389370
2,2,Do you enjoy his...,Education & Know...,GPT-4 generated,"Media, Technolog...",Binary Choice,NaN,"Sure, especially...","Yes, history is ...",British: Do you ...,...,None,None,0,0.204428,0.446994,NaN,0.000000,0.000000,0.204428,0.446994
3,3,As I read some s...,D. Politics and ...,GAS,Politics and Gov...,Likert Scale,"['Approve', 'Dis...",Approve,Approve,British: As I re...,...,Disapprove,Disapprove,2,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
4,4,Most people cons...,A. Social values...,WVS,"Society, Culture...",Likert Scale,"['Freedom', 'Equ...",Freedom,Freedom,British: Most pe...,...,Freedom,Freedom,5,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000
5,5,Now I'd like you...,N. Economy and work,WVS,"Economy, Health,...",Numerical Scale,['Incomes should...,Incomes should b...,8,British: Now the...,...,6,Incomes should b...,13,NaN,NaN,NaN,0.727273,0.679426,0.727273,0.679426
6,6,Do you read cele...,Entertainment & ...,GPT-4 generated,"Lifestyle, Enter...",Binary Choice,NaN,"Sometimes, it's ...",Not my cup of tea.,British: Do you ...,...,None,None,0,0.348711,0.083487,NaN,0.000000,0.000000,0.348711,0.083487
7,7,Do you listen to...,Entertainment & ...,GPT-4 generated,"Lifestyle, Enter...",Binary Choice,NaN,"Nope, I stream m...",I have a small c...,British: Do you ...,...,None,None,0,0.450259,0.171494,NaN,0.000000,0.000000,0.450259,0.171494
8,8,Here are two sta...,D. Politics and ...,WVS,Politics and Gov...,Multiple Choice,['Protecting env...,Protecting envir...,Protecting envir...,British: Here ar...,...,Protecting envir...,Protecting envir...,6,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
9,9,Do you prefer ci...,Travel & Commute,GPT-4 generated,"Lifestyle, Enter...",Binary Choice,NaN,City breaks for ...,I like a good co...,British: Do you ...,...,None,None,0,0.256089,0.799166,NaN,0.000000,0.000000,0.256089,0.799166


In [91]:
mean_score_us = data.groupby(['category_group'])['combined_score_us'].median()
mean_score_uk = data.groupby(['category_group'])['combined_score_uk'].median()
print(mean_score_us, mean_score_uk)

category_group
Economy, Health, and Environment              0.840008
Lifestyle, Entertainment, and Daily Living    0.371487
Media, Technology, and Education              0.303323
Politics and Governance                       0.780699
Society, Culture, and Relationships           0.572775
Name: combined_score_us, dtype: float64 category_group
Economy, Health, and Environment              0.816058
Lifestyle, Entertainment, and Daily Living    0.394809
Media, Technology, and Education              0.335402
Politics and Governance                       0.816327
Society, Culture, and Relationships           0.671670
Name: combined_score_uk, dtype: float64


In [92]:
mean_score_us = data.groupby(['source'])['combined_score_us'].median()
mean_score_uk = data.groupby(['source'])['combined_score_uk'].median()
print(mean_score_us, mean_score_uk)

source
GAS                                                               0.836538
GPT / https://www.anadventurousworld.com/usa-trivia-questions/    0.932235
GPT / https://www.beelovedcity.com/england-quiz                   0.901694
GPT-4 generated                                                   0.392604
WVS                                                               0.727273
Name: combined_score_us, dtype: float64 source
GAS                                                               0.816327
GPT / https://www.anadventurousworld.com/usa-trivia-questions/    0.755683
GPT / https://www.beelovedcity.com/england-quiz                   0.888049
GPT-4 generated                                                   0.393433
WVS                                                               0.800000
Name: combined_score_uk, dtype: float64


In [89]:
mean_score_us = data.groupby(['question type'])['combined_score_us'].median()
mean_score_uk = data.groupby(['question type'])['combined_score_uk'].median()
print(mean_score_us, mean_score_uk)

question type
Binary Choice      0.396288
Likert Scale       0.804483
Multiple Choice    0.438662
Numerical Scale    0.727273
Ordinal Scale      0.545295
Quiz question      0.902448
Name: combined_score_us, dtype: float64 question type
Binary Choice      0.342251
Likert Scale       0.816614
Multiple Choice    0.394809
Numerical Scale    0.679426
Ordinal Scale      0.281433
Quiz question      0.882672
Name: combined_score_uk, dtype: float64


In [ ]:
"""
Case A: Options available
- Multiple Choice
-> if match then 1 else 0
- Likert Scale
-> if match then 1 else diff/len 

- Binary Choice
-> if match then 1 else 0




"""



